https://adventofcode.com/2022/day/19

In [6]:
import re
from collections import Counter

In [2]:
with open("data/19.txt") as fh:
    data = fh.read()

In [3]:
testdata = """\
Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian
"""

In [4]:
def load_data(data):
    ORE, CLY, OBS, GEO = [0, 1, 2, 3]
    L = []
    for line in data.strip().splitlines():
        bp = [[0, 0, 0, 0] for _ in range(4)]
        _, bp[ORE][ORE], bp[CLY][ORE], bp[OBS][ORE], bp[OBS][CLY], bp[GEO][ORE], bp[GEO][OBS] = (int(x) for x in re.findall(r"\d+", line))
        L.append(bp)

    return L

In [5]:
bps = load_data(testdata)
bps[0]

[[4, 0, 0, 0], [2, 0, 0, 0], [3, 14, 0, 0], [2, 0, 7, 0]]

In [11]:
def dfs(bp, max_time=24):
    inds = ORE, CLY, OBS, GEO = [0, 1, 2, 3]
    max_minerals = [0, 0, 0, 0]
    for bot in inds:
        max_minerals[bot] = max(req[bot] for req in bp)

    visited = {}
    stack = [((1,), max_time, [1, 0, 0, 0], [0, 0, 0, 0])]
    max_geo = 0
    while stack:
        pth, tr, bs, ms = stack.pop()
        if tr <= 0:
            continue

        max_geo = max(max_geo, ms[GEO] + bs[GEO] * tr)
        if pth in visited:
            continue
        visited[pth] = (tr, bs, ms)

        # Can we beat best so far even if we make a new geode bot every minute going forward?
        max_poss_geo = ms[GEO] + bs[GEO] * tr + sum(range(tr))
        if max_poss_geo < max_geo:
            visited[pth] = -1
            continue

        for bi in inds:
            # Do we need this bot?
            if bi != GEO and max_minerals[bi] * tr <= ms[bi] + bs[bi] * tr:
                visited[pth + (bi,)] = -2
                continue

            # Do precursor bots exist yet?
            reqs = bp[bi]
            if not all(x for x,y in zip(bs, reqs) if y):
                visited[pth + (bi,)] = -3
                continue

            tr1, bs1, ms1 = tr, bs.copy(), ms.copy()
            # If not all required minerals are present, wait for them to be produced
            while any(a-b < 0 for a,b in zip(ms1, reqs)):
                tr1 -= 1
                ms1 = [a+b for a,b in zip(bs1, ms1)]
            ms1 = [a-b for a,b in zip(ms1, reqs)]
            tr1 -= 1
            ms1 = [a+b for a,b in zip(bs1, ms1)]
            bs1[bi] += 1
            stack.append((pth+(bi,), tr1, bs1, ms1))

    print(max_geo, len(visited), Counter(v for v in visited.values() if v in [-1, -2, -3]))

    return max_geo


In [12]:
%%time
bps = load_data(testdata)
dfs(bps[0])

9 36712 Counter({-1: 16944, -2: 9352, -3: 592})
CPU times: user 225 ms, sys: 0 ns, total: 225 ms
Wall time: 223 ms


9

In [13]:
%%time
dfs(bps[1])

12 64341 Counter({-2: 31375, -1: 15015, -3: 110})
CPU times: user 269 ms, sys: 0 ns, total: 269 ms
Wall time: 272 ms


12

In [14]:
%%time
bps = load_data(data)
ttl = 0
for bpid, bp in enumerate(bps, 1):
    geodes = dfs(bp)
    ttl += bpid * geodes
print(ttl)

0 9838 Counter({-2: 5606, -3: 298})
0 15895 Counter({-2: 9488, -3: 216})
1 19206 Counter({-2: 8270, -1: 3495, -3: 568})
5 22453 Counter({-2: 9463, -1: 6326, -3: 147})
3 11451 Counter({-2: 4445, -1: 3381, -3: 236})
2 20883 Counter({-2: 10824, -1: 2370, -3: 429})
0 11352 Counter({-2: 6518, -3: 298})
1 16347 Counter({-2: 8627, -1: 1880, -3: 159})
14 45585 Counter({-2: 21583, -1: 11675, -3: 47})
0 38672 Counter({-2: 23467, -3: 444})
1 8444 Counter({-2: 3745, -1: 1439, -3: 298})
0 8236 Counter({-2: 4569, -3: 324})
5 35712 Counter({-2: 18405, -1: 6511, -3: 158})
6 14788 Counter({-2: 6480, -1: 4004, -3: 108})
0 15025 Counter({-2: 8826, -3: 220})
0 7577 Counter({-2: 4151, -3: 336})
1 15469 Counter({-2: 9015, -1: 861, -3: 159})
0 19380 Counter({-2: 11604, -3: 444})
15 22641 Counter({-1: 8731, -2: 7792, -3: 120})
6 64055 Counter({-2: 30809, -1: 13289, -3: 450})
0 21605 Counter({-2: 13176, -3: 190})
5 15856 Counter({-1: 6249, -2: 4703, -3: 418})
0 21868 Counter({-2: 12335, -3: 595})
0 35155 Count

In [15]:
%%time
L = []
for bp in bps[:3]:
    geodes = dfs(bp, max_time=32)
    print(geodes)
    L.append(geodes)

11 800000 Counter({-2: 341957, -1: 235126, -3: 619})
11
13 1236825 Counter({-2: 529785, -1: 364099, -3: 364})
13
21 1243611 Counter({-1: 561191, -2: 363900, -3: 1255})
21
CPU times: user 14.9 s, sys: 312 ms, total: 15.2 s
Wall time: 15.2 s


In [11]:
L[0] * L[1] * L[2]

3003